Suguru Maker/analisys/solver/grapher

Suguru is a sudoku like game where you solve a n*p grid filled by 1 - k size zones containing number from 1 to the zone size. 

a number cannot be touching (meaning in a 3*3 grid centered on a number, that number cannot be present again)

In [2]:
import numpy as np
import operator
import random

Suguru Maker

In [3]:
def is_valid(index : tuple, shape : tuple):

    return (0 <= index[0] < shape[0] and
            0 <= index[1] < shape[1])

In [4]:
def get_neighboors(position :tuple, shape : tuple):
    neighboors = []
    # adds position in neighboors remove how ? 
    for i in range(-1, 2):
        for j in range(-1, 2):
            if is_valid((position[0]+i,position[1]+j),shape):
                neighboors.append((position[0]+i,position[1]+j))
    neighboors.remove(position)
    return neighboors


In [ ]:
print([0,1]+,p.array([0,1]))

[0, 1, 0, 1]


In [ ]:
def get_positions_snail(positions :np.array , grid : np.array, shape :tuple):
    old_pos = positions[0]
    
    pos = old_pos

In [7]:
def get_neighboor_line(prev :tuple , grid : np.array, shape :tuple):
    # lots of optimizations possible here just general architecture useless calculus and verif but who cares
    # goes right then down 
    val = np.copy(prev)
    val[1] +=1 
    print("val",val)

    if is_valid(val,shape):
        #checks if this lands in the suguru grid

        if not(grid[*val]):
            #checks if there is a value assigned there
            return val
    else:
        val[1] = 0

        if not(grid[*val]):
            return val
    val[1] = prev[1]
    val[0] += 1 
    if is_valid(val,shape):
        if not(grid[*val]):
            return val
    else:
        val[0] = 0
        if not(grid[*val]):
            return val
    print("ERROR")
    return prev
    




In [8]:
def get_value(suguru: np.array, neighboor:tuple):
    return
    

In [9]:
def get_possibilities(numbers : list):
    #check the values of the neighboors to know which numbers can be added

    ###NOT MATHEMATICALLY 
    
    #we can add a check if all the numbers under present exist in a 3d radius
    
    possibilities = []
    for value in range(1,6):
        if not (value in numbers):
            if value != 1 :
                if value-1 in numbers:
                    possibilities.append(value)
            else:
                possibilities.append(value)
    #can add checkers to check if that value is in a sufficient number ( as there logically cant be more 1 than 5) 
    return possibilities


In [10]:
def get_value(possibilities : np.array):
    #we can try equiprob since we only add a 5 in poss if a 4 is present
    return random.choice(possibilities)        
        


In [11]:
def path_type(method):
    methods = ["line" ,"snail"]
    if method in methods:
        return method
    else : 
        print("unnamed method switching to line")
        return "line"

In [12]:
def make_numbers(suguru : np.array):
    
    #writes the numbers first hopefully there wont be impossible grids? 
    grid_shape = np.shape(suguru)
    grid_size = grid_shape[0]*grid_shape[1]
    #useless for now ? 
    # average_zone_size = 4
    # zone_number = np.ceil(np.size(suguru)/average_zone_size)

    # pick a random spot throw a 1
    # adding a -1 here doesnt change a thing factually just makes it easier to encode snail for now
    # slows very very very slightly by adding 2 operations in the whole calculus  
    start_x = np.random.randint(grid_shape[0]-1)
    start_y = np.random.randint(grid_shape[1]-1)

    suguru[start_x][start_y] = 1

    grid = np.zeros(grid_shape)
    #grid gets allocated positions
    # allocation_type = "line"
    # technique = "line"
    technique = "snail"
    technique = path_type(technique)
    i = 0
    position = (start_x, start_y)
    positions = np.zeros((64,2)).astype(int)
    positions[0] = position

    if technique == "line":
        for i in range (1,64):
            positions[i] = (get_neighboor_line(positions[i-1], grid, grid_shape)) 
            grid[*positions[i]] = 1
    if technique == "snail" : 
        positions = get_positions_snail(positions, grid, grid_shape)
        

        
    """
    while i<grid_size:
        
        # position = get_neighboor_line(position,grid, grid_shape)

    #pick a neighboor
    #assess possible values (if nothing possible here i-e 1-5 is taken then what?) what if snail shape allocation ? should work since no more than 4 neighbors max ? 
    #can do snail-like allocation from random startr point technically #most arranged allocation techniques should work, while random based ones are unsafe
    #distribution will be weighted to have more 1s in the firstly elected part likely ?

        neighboors = get_neighboors(position, grid_shape)
        numbers = []
        for neighboor in neighboors:
            numbers.append(int(suguru[neighboor]))
        possibilities = get_possibilities(numbers)
        value = get_value(possibilities)
        suguru[position] = value
        grid[position] = 1

    #allocate values
    
        i+=1
    """
    print(suguru)

    
    

In [13]:
def make_suguru(size = 8):
    # makes a Filled Suguru grid 
    if 2 < size < 14 : 
        suguru = np.zeros((size,size)).astype(int)
        suguru = make_numbers(suguru)
    else:
         suguru = np.zeros
    return suguru

In [14]:
make_suguru(8)

[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]
